In [18]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras


In [2]:
# 데이터 불러오기
file_path = 'meal_dataset_example (2).csv'
data = pd.read_csv(file_path, encoding='cp949')

In [3]:
# 입력 (독립) 변수와 출력 (종속) 변수 분리
X = data[['메뉴명', '칼로리', '탄수화물', '단백질', '지방', '요일', '날씨', '학교행사', '선호도', '제공인원']]
y = data[['잔반량']]

In [4]:
# 범주형 변수 원핫 인코딩
X_encoded = pd.get_dummies(X, columns=['메뉴명', '요일', '날씨', '학교행사'])

In [5]:
# 수치형 변수 정규화
scaler = StandardScaler()
num_cols = ['칼로리', '탄수화물', '단백질', '지방', '선호도', '제공인원']
X_encoded[num_cols] = scaler.fit_transform(X_encoded[num_cols])

In [6]:
# 학습/검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [7]:
# 모델 정의
input_layer = tf.keras.layers.Input(shape=[X_encoded.shape[1]])
dense = tf.keras.layers.Dense(10, activation='relu')(input_layer)
output = tf.keras.layers.Dense(1)(dense)
model = tf.keras.models.Model(inputs=input_layer, outputs=output)

In [ ]:
# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=1000, verbose=0)
model.fit(X_train, y_train, epochs=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 33737.7969


: 

In [9]:
# 예측 예시
print("예측 결과 (테스트셋 일부):")
print(model.predict(X_test[:5]))
print("실제 잔반량:")
print(y_test[:5].values)

예측 결과 (테스트셋 일부):
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 741ms/step
[[1.0360899 ]
 [0.11951675]]
실제 잔반량:
[[80]
 [90]]


In [10]:
# 새 데이터 예측 예시
new_data = pd.DataFrame([{
    '메뉴명': '제육볶음',
    '칼로리': 600,
    '탄수화물': 50,
    '단백질': 30,
    '지방': 20,
    '요일': '수요일',
    '날씨': '맑음',
    '학교행사': '없음',
    '선호도': 4.0,
    '제공인원': 200
}])

In [11]:
# 새 데이터 전처리
new_data_encoded = pd.get_dummies(new_data)

In [12]:
# 누락된 컬럼 채우기 (기존 학습 데이터 기준)
new_data_encoded = new_data_encoded.reindex(columns=X_encoded.columns, fill_value=0)
new_data_encoded[num_cols] = scaler.transform(new_data_encoded[num_cols])

In [13]:
# 예측
predicted_leftover = model.predict(new_data_encoded)
print("예측된 잔반량:", predicted_leftover[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step
예측된 잔반량: 0.04907676


In [ ]:
# import ipywidgets as widgets
# from IPython.display import display, clear_output

# # 메뉴, 요일, 날씨, 행사 선택지
# 메뉴목록 = data['메뉴명'].unique().tolist()
# 요일목록 = data['요일'].unique().tolist()
# 날씨목록 = data['날씨'].unique().tolist()
# 행사목록 = data['학교행사'].unique().tolist()

# # 위젯 정의
# menu_widget = widgets.Dropdown(options=메뉴목록, description='메뉴명:')
# day_widget = widgets.Dropdown(options=요일목록, description='요일:')
# weather_widget = widgets.Dropdown(options=날씨목록, description='날씨:')
# event_widget = widgets.Dropdown(options=행사목록, description='행사:')

# cal_widget = widgets.IntSlider(value=600, min=100, max=1000, step=10, description='칼로리:')
# carb_widget = widgets.IntSlider(value=50, min=0, max=150, step=5, description='탄수화물:')
# protein_widget = widgets.IntSlider(value=30, min=0, max=100, step=5, description='단백질:')
# fat_widget = widgets.IntSlider(value=20, min=0, max=100, step=5, description='지방:')
# preference_widget = widgets.FloatSlider(value=4.0, min=1.0, max=5.0, step=0.1, description='선호도:')
# served_widget = widgets.IntSlider(value=200, min=1, max=1000, step=10, description='제공인원:')

# predict_button = widgets.Button(description='잔반량 예측하기', button_style='success')
# output_area = widgets.Output()

# # 예측 함수
# def predict_callback(b):
#     with output_area:
#         clear_output()

#         # 입력값 DataFrame 구성
#         input_df = pd.DataFrame([{
#             '메뉴명': menu_widget.value,
#             '칼로리': cal_widget.value,
#             '탄수화물': carb_widget.value,
#             '단백질': protein_widget.value,
#             '지방': fat_widget.value,
#             '요일': day_widget.value,
#             '날씨': weather_widget.value,
#             '학교행사': event_widget.value,
#             '선호도': preference_widget.value,
#             '제공인원': served_widget.value
#         }])

#         # 원핫 인코딩 및 정규화
#         input_encoded = pd.get_dummies(input_df)
#         input_encoded = input_encoded.reindex(columns=X_encoded.columns, fill_value=0)
#         input_encoded[num_cols] = scaler.transform(input_encoded[num_cols])

#         # 예측
#         prediction = model.predict(input_encoded)
#         print(f"📊 예측된 잔반량: {prediction[0][0]:.2f} kg")

# # 버튼에 이벤트 연결
# predict_button.on_click(predict_callback)

# # UI 출력
# display(widgets.VBox([
#     menu_widget, day_widget, weather_widget, event_widget,
#     cal_widget, carb_widget, protein_widget, fat_widget,
#     preference_widget, served_widget,
#     predict_button, output_area
# ]))

In [22]:
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("my_h5_model.keras")

reconstructed_model = keras.models.load_model("my_h5_model.keras")


c:\Users\syjsn\OneDrive\ドキュメント\개인폴더\파이썬\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
